In [ ]:
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import numpy as np


src_root = Path().resolve()
list(src_root.iterdir())

In [ ]:
default_img_path = src_root / "data" / "maze.png"
def load_photo(img_path = default_img_path):
    image = cv2.imread(str(img_path))
    return image

In [ ]:
maze_path_manual = load_photo(src_root / "data" / "maze_path_manual.png")

In [ ]:
mini_maze_path = cv2.resize(maze_path_manual, None, fx=0.1, fy=0.1, interpolation=cv2.INTER_LINEAR)

In [ ]:
def viz(cv_img):
    plt.imshow(cv_img)
    plt.show()

viz(maze_path_manual)

In [ ]:
viz(maze_path_manual[:, :, 2])

In [ ]:
viz(mini_maze_path)

In [ ]:
from typing import Set
bug_counter = 0

def get_unvisited_neighbours(element, matrix_shape, visited_set, frontier_set):
    # assume 2D matrix and that we are checking immediate neighbours - a 3*3 receptive field
    x_current, y_current = element
    x_end, y_end = matrix_shape
    
    # use +2 for upper bound to work around range behaviour at upper limits
    x_range = range(max(x_current-1,0),min(x_current+2,x_end))
    y_range = range(max(y_current-1,0),min(y_current+2,y_end))
    
    elements_to_visit = []
    for x in x_range:
        for y in y_range:
            if x == x_current and y == y_current:
                # do not readd the current node
                continue
            elif (x, y) in frontier_set:
                # do not schedule for multiple visits
                continue
            elif (x, y) not in visited_set:
                elements_to_visit.append((x, y))
            else:
                pass
    assert len(elements_to_visit) < 9, f"Too many neighbouring elements returned for a 2D pixel! {len(elements_to_visit)}" 
    return elements_to_visit


def create_fitness_score(element, difficulty_map, output_values):
    # assume 2D matrix and that we are checking immediate neighbours - a 3*3 receptive field
    global bug_counter
    x_current, y_current = element
    x_end, y_end = output_values.shape
    
    x_range = range(max(x_current-1,0),min(x_current+1,x_end))
    y_range = range(max(y_current-1,0),min(y_current+1,y_end))

    # distance penalty: lower on the prescribed path:
    # RGB -> red is idx 0
    is_red = bool(difficulty_map[x_current, y_current, 0])
    is_green = bool(difficulty_map[x_current, y_current, 1])
    is_blue = bool(difficulty_map[x_current, y_current, 2])
    if is_blue:
        penalty = 0
    elif is_red or is_green:
        penalty = 0.001
    else:
        penalty = 1
    
    # take the minimum score of any neighbour then add the distance penalty
    # use filter to remove None i.e. unvisited neighbours. These can be ignored as 
    # they will always be more expensive than earlier nodes.        
    
    element_score = None
    for x in x_range:
        for y in y_range:
            if output_values[x, y] != np.inf:  # 0 is valid
                candidate_score = output_values[x, y] + penalty
                if element_score is None or element_score > candidate_score:
                    # update
                    element_score = candidate_score
                    if x == x_current and y == y_current:
                        print(f"found best value already in place! This should happen once. {bug_counter}")
                        if bug_counter > 0:
                            breakpoint
                        bug_counter = bug_counter + 1
                    
    assert element_score != np.inf
    assert element_score is not None
    return element_score


from tqdm import tqdm, trange
from functools import reduce

def generic_search_algorithm(frontier, visited_set: Set, difficulty_map, output_values):
    # frontier_set = set(frontier)
    matrix_shape = output_values.shape
    
    # tqdm progress bar up to the number of pixels
    total_pixels = reduce(lambda x, y: x*y,matrix_shape, 1)
    print(total_pixels)
    progress = trange(total_pixels)
    progress_iterator = iter(progress)
    
    for element in frontier:
        # frontier_set.remove(element)  # actually not necessary - used as a preset for visited status
        # print(element)
        output_values[element] = create_fitness_score(element, difficulty_map, output_values)
        
        # Visit complete, come again soon!
        visited_set.add(element)
        
        unvisited_neighbours = get_unvisited_neighbours(element, matrix_shape, visited_set, frontier)
        
        # visited_set[element] = 1
        # breadth first search - elements are added to the back of the frontier queue
        # [frontier_set.add(neighbour) for neighbour in unvisited_neighbours]
        frontier.extend(unvisited_neighbours)
        # print(f"Frontier size: {len(frontier)}")
        # print(f"Visited Set size: {len(visited_set)}")
        
        next(progress_iterator)
    progress.close()
    
    return output_values


def make_naive_graph_path(maze_path_img):
    
    # start with the blue endzone
    endzone = maze_path_img[:, :, 2]
    # Step 1: Prepare a blank image
    score_matrix = np.full(endzone.shape[:2], dtype="float64", fill_value=np.inf)  # Creating a black canvas
    # score_matrix = np.empty(endzone.shape[:2], dtype="float64", cval=np.inf)  # Creating a black canvas
    # visited_matrix = np.zeros(endzone.shape[:2], dtype="uint8")  # Creating a black canvas
    
    # = np.full((2, 2), None)  # np.inf

    endzone_xs, endzone_ys = np.where(endzone)
    # any pixel in the endzone will do
    seed_pixel_coords = (endzone_xs[0], endzone_ys[0])
    score_matrix[seed_pixel_coords] = 0
    # visited_matrix[seed_pixel_coords] = 1
    
    visited_set = set()
    # Make the frontier a dictionary so we can check membership efficiently and also keep the order of appendage
    # frontier = defaultdict()
    # frontier[seed_pixel_coords]=None

    # print(seed_pixel_coords)
    scored_pixels = generic_search_algorithm(frontier=[seed_pixel_coords], visited_set=visited_set, difficulty_map=maze_path_img, output_values=score_matrix)
    
    return scored_pixels 

# gradient_map = make_naive_graph_path(maze_path_manual)
gradient_map = make_naive_graph_path(mini_maze_path)
viz(gradient_map)

In [ ]:
480 * 640

In [ ]:
# gradient_map_scaled = gradient_map * 255.0/gradient_map.max()

# Normalize image to between 0 and 255
# gradient_map_scaled = gradient_map/(gradient_map.max()/1024.0)


In [ ]:
import pandas as pd
pd.DataFrame(gradient_map)

In [ ]:
gradient_map.max()

In [ ]:
viz(gradient_map)

In [ ]:
endzone = maze_path_manual[:, :, 2]

In [ ]:
maze_path_manual.shape